### 1. 为什么需要Runtime Configuration
在实际应用中，经常遇到需求：同一个Graph，在不同的调用场景下，需要使用不同的配置参数。
比如：
- 有时候我们希望Graph使用不同的LLM模型
- 有时候我们希望Graph使用不同的Prompt模板
- 有时候我们希望根据不同的用户或环境，动态调整Graph的行为参数
如果每次都要重新定义Graph或者修改代码，就太麻烦了。这时候，Runtime Configuration（运行时配置）就派上用场了。
Runtime Configuration 允许我们在调用Graph时，动态地传递配置参数，而不需要修改Graph的定义或代码。
### 2. Runtime Configuration的基本设置方法
增加运行时设置的基本步骤包括：
1. 指定一个关于运行时配置的schema；
2. 将这个schema写入节点函数或者条件边的路由函数中；
3. 在搭建Graph时与state一起传入StateGraph中；

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langgraph.runtime import Runtime

# 特定配置的schema
# 负责定义运行时配置的结构，告诉LangGraph，我们在调用Graph时可以传入哪些配置参数
class ContextSchema(TypedDict):
    my_runtime_value: str

class State(TypedDict):
    my_state_value: str

# 在节点函数中配置runtime参数
# 首先需要从langgraph.runtime中导入Runtime类型
# Runtime是一个泛型类型，我们在指定自定义的ContextSchema时，需要使用方括号
# 然后，在节点函数的参数中添加runtime参数，将它的类型设定为Runtime[ContextSchema]
# 这样，节点函数就可以访问运行时配置了。
def node(state: State, runtime: Runtime[ContextSchema]):
    if runtime.context["my_runtime_value"] == "a":
        return {"my_state_value": "1"}
    elif runtime.context["my_runtime_value"] == "b":
        return {"my_state_value": "2"}
    else:
        raise ValueError("unknown values.")

# 构建Graph，向Graph中传入ContextSchema
# 关键点在于StateGraph的初始化，除了传入State，还需要传入context_schema参数
# 这样，LangGraph就会在调用Graph时，将运行时配置传递给节点函数
builder = StateGraph(State, context_schema=ContextSchema)
builder.add_node("node", node)
builder.add_edge(START, "node")
builder.add_edge("node", END)
graph = builder.compile()
# Runtime Configuration的使用
print(graph.invoke({}, context={"my_runtime_value": "a"}))

{'my_state_value': '1'}


### 3. Runtime Configuration的实际应用
#### 3.1 指定运行时使用的Model
在实际开发中，我们可能需要根据不同的场景，使用不同的Model。这时，就可以利用Runtime Configuration来实现。

In [3]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.runtime import Runtime
from typing import TypedDict

load_dotenv()

class ContextSchema(TypedDict):
    model_type: str

MODELS = {
    "qwen": init_chat_model(model="Qwen/Qwen3-8B", model_provider="openai"),
    "deepseek": init_chat_model(model="deepseek-ai/DeepSeek-V3.2", model_provider="openai"),
}

def call_model(state: MessagesState, runtime: Runtime[ContextSchema]):
    """获取运行时配置中的model_type，根据其初始化对应的Model"""
    model_type = (runtime.context or {}).get("model_type", "qwen")
    # 从运行时配置中获取model_type，默认使用qwen
    model = MODELS[model_type]
    response = model.invoke(state["messages"])
    return {"messages": [response]}

# 搭建Graph传入context_schema
builder = StateGraph(MessagesState, context_schema=ContextSchema)
builder.add_node("node", call_model)
builder.add_edge(START, "node")
builder.add_edge("node", END)
graph = builder.compile()

# 使用
input_message = {"role": "user", "content": "你好"}
response_1 = graph.invoke({"messages": [input_message]}, context={})["messages"][-1]
print(response_1)
response_2 = graph.invoke({"messages": [input_message]}, context={"model_type": "deepseek"})["messages"][-1]
print(response_2)

print(response_1.response_metadata["model_name"])
print(response_2.response_metadata["model_name"])


content='你好！😊 有什么我可以帮你的吗？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '019ca2c30091ea69b09561a86ea9c2ab', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019ca2c2-ff6b-7b83-9863-c56d9c64bdf3-0' usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}}
content='你好！👋 很高兴见到你！有什么我可以帮助你的吗？无论是回答问题、提供建议，还是闲聊，我都很乐意和你交流～ 😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 5, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None,

#### 3.2 指定运行时使用的System Prompt
除了动态切换模型，Runtime Configuration的另一个常见的应用场景是动态指定System Prompt。
比如，同一个客服Agent，在处理技术问题时需要使用技术支持的System Prompt，在处理退款问题时需要使用客服的System Prompt。如果每次都要重新定义Graph，显然不够灵活。

In [4]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.runtime import Runtime
from typing_extensions import TypedDict
from langchain_core.messages import SystemMessage

# 定义Runtime Configuration的Schema
class ContextSchema(TypedDict):
    system_prompt: str

# 定义处理System Prompt的节点函数
def call_model_with_prompt(state: MessagesState, runtime: Runtime[ContextSchema]):
    """获取运行时配置中的system_prompt，根据其初始化对应的Model"""
    model = init_chat_model(
        model="Qwen/Qwen3-8B",
        model_provider="openai",
    )
    system_prompt = (runtime.context or {}).get("system_prompt", "You are a helpful assistant.")
    system_message = SystemMessage(content=system_prompt)
    # 将 System Message添加到消息列表的开头
    messages = [system_message] + state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

# 搭建Graph
builder = StateGraph(MessagesState, context_schema=ContextSchema)
builder.add_node("model", call_model_with_prompt)
builder.add_edge(START, "model")
builder.add_edge("model", END)
graph = builder.compile()

# 使用
input_message = {"role": "user", "content": "python是什么"}
# 使用默认的system prompt
reponse_1 = graph.invoke({"messages": [input_message]}, context={})
print(reponse_1)

response_2 = graph.invoke({"messages": [input_message]}, context={"system_prompt": "你是一个专业的python助手"})
print(response_2)


{'messages': [HumanMessage(content='python是什么', additional_kwargs={}, response_metadata={}, id='d409ff1f-eb28-46d0-9a03-907b110213a7'), AIMessage(content='Python 是一种**高级的、解释型的、通用的**编程语言，由 **Guido van Rossum** 于 1991 年首次发布，名字来源于英国喜剧团体 *Monty Python*。Python 的设计理念强调代码的**可读性**和**简洁性**，这使得它成为初学者和专业人士都广泛使用的语言之一。\n\n---\n\n### ✅ Python 的特点包括：\n\n1. **简洁易读**  \n   Python 的语法结构不像 C++、Java 那样繁琐，使用缩进代替大括号，代码更清晰易读。\n\n2. **跨平台性**  \n   Python 可以在多种操作系统上运行，如 Windows、Linux、macOS 等。\n\n3. **面向对象**  \n   Python 支持面向对象编程（OOP），同时也支持过程式编程。\n\n4. **动态类型**  \n   变量不需要预先声明类型，类型在运行时自动识别。\n\n5. **开源**  \n   Python 是开源的，拥有强大的社区支持，有很多免费的库和框架。\n\n6. **广泛应用**  \n   Python 被广泛用于：\n   - **Web 开发**（如 Django、Flask）\n   - **数据科学**（如 Pandas、NumPy、Matplotlib、Scikit-learn）\n   - **人工智能与机器学习**（如 TensorFlow、PyTorch、Keras）\n   - **自动化脚本**（如爬虫、系统管理）\n   - **科学计算**（如 SciPy）\n   - **游戏开发**（如 Pygame）\n   - **网络爬虫**、**数据分析**、**图形界面开发**、**数据库编程**等。\n\n---\n\n### 📌 Python 的语法示例：\n\n```python\n# 打印一句话\nprint("Hello, World!")\n\n# 定